# Fit harmonics to filtered $A_n$ data

In [ ]:
import pymc3 as pm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime
import h5py

#from soda.dataio.conversion import readotps

from scipy.optimize import fmin_powell, fmin, fmin_cg, fmin_ncg

from theano import shared
from theano import tensor as tt
import arviz as az

from soda.utils.timeseries import timeseries, skill, rmse
from soda.utils.uspectra import uspectra, getTideFreq
from soda.utils.othertime import SecondsSince, TimeVector
from mycurrents import oceanmooring as om
from soda.utils.harmonic_analysis import harmonic_fit_array
from soda.utils.mysignal import power_spectra

from scipy import signal

from tqdm import tqdm

import matplotlib as mpl

In [ ]:
%matplotlib notebook

In [ ]:
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 'large'

In [ ]:
# ncfile = '/home/suntans/Share/ARCHub/DATA/FIELD/browse-basin-kissme/Data/NetCDF/KISSME_Fitted_Buoyancy_wout_motion.nc'
ncfile = '/home/suntans/Share/ARCHub/DATA/FIELD/ShellCrux/KP150_Fitted_Buoyancy_wout_motion.nc'
# ncfile = r'C:\Projects\ARCHub\DATA\FIELD\ShellCrux\KP150_Fitted_Buoyancy_wout_motion.nc'
ds1 = xr.open_dataset(ncfile,group='KP150_phs1')
ds2 = xr.open_dataset(ncfile,group='KP150_phs2')
ds1

In [ ]:
# subsample = 60
subsample = 20


In [ ]:
# Filter each time series and concatenate
A1 = om.OceanMooring(ds1.time.values, ds1['A_n'][:,0],0.0)
A2 = om.OceanMooring(ds2.time.values, ds2['A_n'][:,0],0.0)

A1f = om.OceanMooring(A1.t, A1.filt((34*3600, 6*3600), btype='band'), 0.0)
A2f = om.OceanMooring(A2.t, A2.filt((34*3600, 6*3600), btype='band'), 0.0)

A_n = A1f.concat(A2f)
A_n_1h_6 = xr.DataArray(A_n.y[::subsample], dims=('time'), coords={'time':A_n.t[::subsample]})

A1f = om.OceanMooring(A1.t, A1.filt((34*3600, 3*3600), btype='band'), 0.0)
A2f = om.OceanMooring(A2.t, A2.filt((34*3600, 3*3600), btype='band'), 0.0)


A_n = A1f.concat(A2f)
A_n_1h = xr.DataArray(A_n.y[::subsample], dims=('time'), coords={'time':A_n.t[::subsample]})



# A_n_1h.loc['2016-09-15':'2016-10-31']=np.nan

#A_n_1h = A_n_1h.sel(time=slice('2016-11-02','2017-05-02'))

In [ ]:
plt.figure()
A_n_1h.plot(lw=0.25)
A_n_1h_6.plot(lw=0.2)


In [ ]:
def sine_model(beta_s, ff, t):
    n = len(ff)

    result = beta_s[0]*np.ones_like(t)
    for ii in range(0,n):
        result += beta_s[2*ii+1]*np.cos(ff[ii] * t) + beta_s[2*ii+2]*np.sin(ff[ii]*t)

    return result

def sine_model_pm(beta_s, ff, t):
    n = len(ff)

    result = beta_s[0] + beta_s[1]*t
    for ii in range(0,n):
        result += beta_s[2*ii+2]*pm.math.cos(ff[ii] * t) + beta_s[2*ii+3]*pm.math.sin(ff[ii]*t)

    return result

def sine_model_notrend_pm(beta_s, ff, t):
    n = len(ff)
    
    result = beta_s[0]+0*t
    for ii in range(0,n):
        result += beta_s[2*ii+1]*pm.math.cos(ff[ii] * t) + beta_s[2*ii+2]*pm.math.sin(ff[ii]*t)

    return result
    
def cosine_model_pm(beta_s, ff, t):
    n = len(ff)
    
    result = beta_s[0]+0*t
    for ii in range(0,n):
        result += beta_s[2*ii+1]*pm.math.cos(ff[ii] * t - beta_s[2*ii+2])

    return result

def sine_model_notrend(beta_s, ff, t):
    n = len(ff)
    
    result = beta_s[0] + 0*t
    for ii in range(0,n):
        result += beta_s[2*ii+1]*np.cos(ff[ii] * t) + beta_s[2*ii+2]*np.sin(ff[ii]*t)

    return result    

def sine_model_envelope(beta_s, ff, t):
    n = len(ff)
    
    #result = t*0
    
    #result = np.zeros(t.shape)
    result = beta_s[0] + 0*t

    for ii in range(0,n):
        result += beta_s[2*ii+1]*np.cos(ff[ii] * t) + beta_s[2*ii+2]*np.sin(ff[ii]*t)
        
    # Compute the imaginary part by adding a 90 degree phase shift
    #result_i = t*0
    result_i = np.zeros(t.shape) # Imaginary mean component is zero

    for ii in range(0,n):
        result_i += beta_s[2*ii+1]*np.cos(ff[ii] * t + np.pi/2) \
            + beta_s[2*ii+2]*np.sin(ff[ii]*t + np.pi/2)
    
    return np.sqrt(result*result + result_i*result_i)

# def sine_model_envelope_pm(beta_s, ff, t):
#     n = len(ff)
    
#     #result = t*0
    
#     result = tt.zeros(t.shape)

#     for ii in range(0,n):
#         result += beta_s[2*ii]*pm.math.cos(ff[ii] * t) + beta_s[2*ii+1]*pm.math.sin(ff[ii]*t)
        
#     # Compute the imaginary part by adding a 90 degree phase shift
#     #result_i = t*0
#     result_i = tt.zeros(t.shape)

#     for ii in range(0,n):
#         result_i += beta_s[2*ii]*pm.math.cos(ff[ii] * t + np.pi/2) \
#             + beta_s[2*ii+1]*pm.math.sin(ff[ii]*t + np.pi/2)
    
#     return pm.math.sqrt(result*result + result_i*result_i)

In [ ]:
# Create a function with similar functionality as harmonic_fit in soda
def harmonic_fit_mcmc(time, X, frq, mask=None, axis=0, basetime=None,         **kwargs):
    """
    Harmonic fitting using Bayesian inference
    """
    tday = 86400.
    # Convert the time to days
    dtime = SecondsSince(time, basetime=basetime )
    
    
    # Generate initvalues from least-squares fitting
    ts = timeseries(time, X)
    amp, phs, _, _, h_lsq, err = ts.tidefit(frqnames=names)
    
    initvals = {'beta_mean':0.}
    initvals = {'sigma':1.}

    ii=-1
    for aa, pp in zip(amp,phs):
        ii+=1
        initvals.update({'beta_amp_%d'%ii:aa})
        initvals.update({'beta_phs_%d'%ii:pp})
    print(initvals)

    
    dtime /= tday
    
    # Convert the frequencies to radians / day
    omega = [ff*tday for ff in frq]
    #omega = frq
    
    # Number of parameters
    n_params = 2*len(omega) + 1
    
    print('Number of Parametrs: %d\n'%n_params, omega)

    with pm.Model() as my_model:
        ###
        # Create priors for each of our variables
        BoundNormal = pm.Bound(pm.Normal, lower=0.0)

        # Mean
        beta_mean = pm.Normal('beta_mean', mu=0, sd=1)
        # Trend
        #beta_linear = pm.Normal('beta_linear', mu=0, sd=1.)

        #beta_s = [beta_mean, beta_linear]
        beta_s=[beta_mean]

        # Harmonics
        for n in range(0,2*len(omega),2):
            beta_s.append(pm.Normal('beta_%d_re'%(n//2), mu=1., sd = 5.))
            beta_s.append(pm.Normal('beta_%d_im'%(n//2), mu=1., sd = 5.))
        #for n in range(0,len(omega)):
        #    beta_s.append(BoundNormal('beta_amp_%d'%n,mu=1., sd=10.))
        #    #beta_s.append(pm.Uniform('beta_amp_%d'%n,lower=0, upper=30))
        #    beta_s.append(pm.Uniform('beta_phs_%d'%n, lower=-np.pi, upper=np.pi))

        ###
        # Generate the likelihood function using the deterministic variable as the mean
        #mu_x = cosine_model_pm(beta_s, omega, dtime)
        #mu_x = pm.Deterministic('mu_x', cosine_model_pm(beta_s, omega, dtime))
        mu_x = sine_model_notrend_pm(beta_s, omega, dtime)

        
        #sigma = BoundNormal('sigma', mu=1.,sd=0.25)
        sigma = pm.HalfNormal('sigma',5.)
        
        #sigma = 1.0
        X_obs = pm.Normal('X_obs', mu=mu_x, sd=sigma, observed=X)
        #X_obs = pm.Normal('X_obs', mu=mu_x, sd=1, observed=X)
        
        mp = pm.find_MAP()
        print(mp)
        # Inference step...
        #step = pm.Metropolis()
        #step = pm.NUTS()
        step = None
        start = None
        trace = pm.sample(500, tune=1000, start = start, step=step, cores=2,
                         )#nuts_kwargs=dict(target_accept=0.95, max_treedepth=16, k=0.5))
    
    
    # Return the trace and the parameter stats
    return trace,  my_model, omega, dtime, h_lsq

In [ ]:
# Create a function with similar functionality as harmonic_fit in soda
def harmonic_fit_mcmc_arn(time, X, frq, arn=1, mask=None, axis=0, basetime=None,         **kwargs):
    """
    Harmonic fitting using Bayesian inference
    
    Model the errors using an auto-regressive model
    """
    tday = 86400.
    # Convert the time to days
    dtime = SecondsSince(time, basetime=basetime )
    nt = dtime.shape[0]
    
    
    # Generate initvalues from least-squares fitting
    ts = timeseries(time, X)
    amp, phs, _, _, h_lsq, err = ts.tidefit(frqnames=names)
    
    initvals = {'beta_mean':0.}
    initvals = {'sigma':1.}

    ii=-1
    for aa, pp in zip(amp,phs):
        ii+=1
        beta_re = amp*np.cos(phs)
        beta_im = amp*np.sin(phs)
        initvals.update({'beta_%d_re'%ii:aa})
        initvals.update({'beta_%d_im'%ii:pp})
    print(initvals)

    
    dtime /= tday
    
    # Convert the frequencies to radians / day
    omega = [ff*tday for ff in frq]
    #omega = frq
    
    # Number of parameters
    n_params = 2*len(omega) + 1
    
    print('Number of Parametrs: %d\n'%n_params, omega)

    with pm.Model() as my_model:
        ###
        # Create priors for each of our variables
        BoundNormal = pm.Bound(pm.Normal, lower=0.0)

        # Mean
        beta_mean = pm.Normal('beta_mean', mu=0, sd=1)
        # Trend
        #beta_linear = pm.Normal('beta_linear', mu=0, sd=1.)

        #beta_s = [beta_mean, beta_linear]
        beta_s=[beta_mean]

        # Harmonics
        for n in range(0,2*len(omega),2):
            beta_s.append(pm.Normal('beta_%d_re'%(n//2), mu=1., sd = 5.))
            beta_s.append(pm.Normal('beta_%d_im'%(n//2), mu=1., sd = 5.))
        #for n in range(0,len(omega)):
        #    beta_s.append(BoundNormal('beta_amp_%d'%n,mu=1., sd=10.))
        #    #beta_s.append(pm.Uniform('beta_amp_%d'%n,lower=0, upper=30))
        #    beta_s.append(pm.Uniform('beta_phs_%d'%n, lower=-np.pi, upper=np.pi))

        ###
        # Generate the likelihood function using the deterministic variable as the mean
        #mu_x = cosine_model_pm(beta_s, omega, dtime)
        #mu_x = pm.Deterministic('mu_x', cosine_model_pm(beta_s, omega, dtime))
        mu_x = sine_model_notrend_pm(beta_s, omega, dtime)

        
        #sigma = BoundNormal('sigma', mu=1.,sd=0.25)
        #sigma = pm.HalfNormal('sigma',5.)
        
        #sigma = 1.0
        #X_obs = pm.Normal('X_obs', mu=mu_x, sd=sigma, observed=X)
        #X_obs = pm.Normal('X_obs', mu=mu_x, sd=1, observed=X)
        
        # Use an autoregressive model for the error term
        beta = pm.Normal('beta', mu=0, sigma=1., shape=arn)
        sigma = pm.InverseGamma('sigma',1,1)

        #err = pm.AR('err', beta, sigma=sigma, shape=nt)

        #sigma = pm.HalfNormal('sigma',1.)
        #X_obs = pm.Normal('X_obs', mu=mu_x+err, sd=sigma, observed=X)
        X_obs = pm.AR('X_obs', beta, sigma=sigma, observed=X - mu_x)
        
        #mp = pm.find_MAP()
        #print(mp)
        
        # Inference step...
        #step = pm.Metropolis()
        #step = pm.NUTS()
        step = None
        start = None
        trace = pm.sample(500, tune=1000, start = start, step=step, cores=2,
                         return_inferencedata=False)#nuts_kwargs=dict(target_accept=0.95, max_treedepth=16, k=0.5))
        
        #trace = None # For testing
        
        #prior = pm.sample_prior_predictive()
        
        # Ouput an arviz object
        #ds_az = az.from_pymc3(prior=prior)

        
    
    
    # Return the trace and the parameter stats
    return trace,  my_model, omega, dtime, h_lsq

In [ ]:
# tidecons = ['M2','S2','N2','K2','K1','O1','P1','Q1','MA2','MB2']
# tidecons = ['M2','S2',] # Non-station harmonics
# # tidecons = ['M2','S2',]
# tidecons_2 = ['N2','K1','O1'] # Stationary harmonics

# frq,names = getTideFreq(tidecons)
# frq2,names = getTideFreq(tidecons_2)


# # Add on long-term modulations
# tdaysec = 86400
# fA = 2*np.pi/(365*tdaysec)
# # fSA = 2*np.pi/(182*tdaysec)
# # fTA = 2*np.pi/(120*tdaysec)
# # f90 = 2*np.pi/(90*tdaysec)

# # longperiods = [0, fA, fSA, fTA]
# #longperiods = [0, 2*np.pi/(365*tdaysec), 2*np.pi/(182*tdaysec), 2*np.pi/(120*tdaysec)]
# # longperiods = [0, 2*np.pi/(182*tdaysec),]
# #longperiods = [0,  2*np.pi/(120*tdaysec), 2*np.pi/(30*tdaysec)]
# # longperiods = [-fTA, -fSA, -fA, 0, fA, fSA, fTA]
# # longperiods = [0, fA, fSA]
# longperiods = [-3*fA, -2*fA, -1*fA, 0, 1*fA, 2*fA, 3*fA]

# # longperiods = [-4*fA, -3*fA, -2*fA, 0, 2*fA, 3*fA, 4*fA]

# sitename = 'M2S2nonstat_N2K1O1'
# #sitename = 'M2S2_lowfreq_90d'
# # outputh5 = '../inputs/a0_samples_harmonicfit_M2S2lowfreq_ATASA_12month.h5'
# outputh5 = '../inputs/a0_samples_harmonicfit_{}_12month.h5'.format(sitename)
# #outputh5 = '../inputs/a0_samples_harmonicfit_M2S2K1O1lowfreq_12month.h5'

# frq_lt = []
# for ff in frq:
#     for ll in longperiods:
#         frq_lt.append(ff+ll)
        
# # Append the stationary harmonics
# for ff in frq2:
#     frq_lt.append(ff)

######

# 
# tidecons = ['M2','S2','N2','K1','O1'] # Non-stationary harmonics
# number_annual_harmonics = 3
# sitename = 'M2S2N2K1O1_nonstat_AR1'

tidecons = ['M2','S2','N2','K1','O1'] # Tidal harmonic
number_annual_harmonics = 0
arn = 4
sitename = 'M2S2N2K1O1_na{}_AR{}_dt20min'.format(number_annual_harmonics, arn)


# tidecons = ['M2','S2',]

frq,names = getTideFreq(tidecons)

# Add on long-term modulations
tdaysec = 86400
fA = 2*np.pi/(365.25*tdaysec)
#longperiods = [-3*fA, -2*fA, -1*fA, 0, 1*fA, 2*fA, 3*fA]

longperiods=[]
for n in range(-number_annual_harmonics, number_annual_harmonics+1):
    longperiods.append(n*fA)

# longperiods = [-4*fA, -3*fA, -2*fA, 0, 2*fA, 3*fA, 4*fA]

# sitename = 'M2S2N2K1O1nonstat'

#sitename = 'M2S2_lowfreq_90d'
# outputh5 = '../inputs/a0_samples_harmonicfit_M2S2lowfreq_ATASA_12month.h5'
# outputh5 = '../inputs/a0_samples_harmonicfit_{}_12month.h5'.format(sitename)

#outputh5 = '../inputs/a0_samples_harmonicfit_M2S2K1O1lowfreq_12month.h5'

frq_lt = []
for ff in frq:
    for ll in longperiods:
        frq_lt.append(ff+ll)
        
######
# sitename = 'M2S2N2K2K1O1P1Q1'
# outputh5 = '../inputs/a0_samples_harmonicfit_{}_12month.h5'.format(sitename)
# tidecons = ['M2','S2','N2','K2','K1','O1','P1','Q1']
# # # tidecons = ['M2','S2','N2','K2','K1','O1','P1','Q1','MA2','MB2']
# frq_lt,names = getTideFreq(tidecons)

#####

basetime = datetime(2016,1,1)

# X_sd = A_n_1h.y.std()
# X_mu = A_n_1h.y.mean()

X_sd = 1
X_mu = 0

idx = ~np.isnan(A_n_1h.values)
X = A_n_1h.values[idx] - X_mu
X /= X_sd

timein = A_n_1h.time.values[idx]

# trace, my_model, omega, dtime, h_lsq = harmonic_fit_mcmc(A_n_1h.time.values[idx], X, frq_lt, basetime=basetime)
trace, my_model, omega, dtime, h_lsq = harmonic_fit_mcmc_arn(timein, X, frq_lt,\
                                                             arn=arn, basetime=basetime)

pm.summary(trace)


In [ ]:
trace[0]

In [ ]:
# Fixed prediction time
rhotime = pd.date_range('2016-05-01','2017-05-02',freq='1H').values
rhotime.shape

In [ ]:
def generate_ar_sample(dtime, omega, trace, use_ar=True):
    true_center = 0.
    T = dtime.shape[0]
    y = np.zeros((T,))
    
    arn = trace['beta'].shape[0]
    
    if use_ar:
        for t in range(arn, T):
            for ii in range(arn):
                y[t] += trace['beta'][ii] * y[t - ii- 1] 
            y[t] += np.random.normal(loc=true_center, scale=trace['sigma'])
    
    betas = [trace['beta_mean']]
    nomega = len(omega)
    for ii in range(nomega):
        betas.append(trace['beta_{}_re'.format(ii)])
        betas.append(trace['beta_{}_re'.format(ii)])
 
    # Add on an oscillatory component
    mu = sine_model_notrend(betas, omega, dtime)
    y+=mu
    
    return y

In [ ]:
# Output the data at a new time
tnew = rhotime
tsecnew = SecondsSince(tnew,basetime=basetime)
tdaynew = tsecnew/86400.

y_ar = []
for tt in trace:
    y_ar.append(generate_ar_sample(tdaynew, omega, tt, use_ar=True))
    
y_no_ar = generate_ar_sample(tdaynew, omega, trace[200], use_ar=False)

a0_pred = np.array(y_ar)

In [ ]:
plt.figure()
plt.plot(dtime, X,'r--',lw=0.5)
plt.plot(tdaynew, y_ar[10],'0.5', lw=0.5,)
plt.plot(tdaynew, y_no_ar,'b',lw=0.5)

In [ ]:
# Convert the data to arviz structure and save

# ds = az.from_pymc3_predictions({'a0':a0_pred})

# Save the predictions
dims = ('chain','draw','time')
ds = az.from_pymc3_predictions({'a0':a0_pred}, \
                coords={'time':tnew,'chain':np.array([1])}, dims={'a0':dims})

# Save the posterior
ds2 = az.from_pymc3(trace=trace)

# Update the observed data becuase it comes out as a theano.tensor in the way
# our particular model is specified
ds2.observed_data['X_obs'] = xr.DataArray(X, dims=('time',), coords={'time':timein})

# This merges the data sets
ds2.extend(ds)
ds2

In [ ]:
# Save the data to netcdf
outputnc = '../inputs/a0_samples_harmonicfit_{}_12month.nc'.format(sitename)
ds2.to_netcdf(outputnc)
print(outputnc)

In [ ]:
# Test loading the data (use xarray)
xr.open_dataset(outputnc,'predictions')

In [ ]:
plt.figure(figsize=(8,6))

ax1=plt.subplot(211)
plt.plot(tnew, np.median(a0_pred,axis=0),'b--')
plt.fill_between(tnew, np.percentile(a0_pred,2.5,axis=0),\
                 np.percentile(a0_pred,97.5,axis=0),color='b',alpha=0.5)

#plt.plot(tnew, trace['err'].mean(axis=0),'k')

A_n_1h.plot(color='r',ls='--',lw=0.5)

plt.subplot(212,sharex=ax1, sharey=ax1)
plt.plot(tnew, np.median(a0_pred,axis=0),'b--')

# plt.plot(tnew, err,'r',lw=0.3)

In [ ]:
plt.figure(figsize=(10,4))

A_n_1h.plot(color='r',ls='--',lw=0.5)

plt.plot(tnew, np.median(a0_pred,axis=0),'b',lw=0.5)
plt.fill_between(tnew, np.percentile(a0_pred,2.5,axis=0),\
                 np.percentile(a0_pred,97.5,axis=0),color='0.5',alpha=0.5)

plt.ylim(-40,40)
plt.xlim(datetime(2017,1,24),datetime(2017,4,7))
# plt.xlim(datetime(2016,7,1),datetime(2016,9,1))

plt.grid(b=True)
plt.xlabel('Date [yyyy-mm]')
plt.ylabel('Amplitude [m]')
plt.tight_layout()


plt.savefig('../FIGURES/a0_harmonic_zoom_{}.png'.format(sitename),dpi=150)
plt.savefig('../FIGURES/a0_harmonic_zoom_{}.pdf'.format(sitename),dpi=150)

In [ ]:
# plot the min/max only
a0_mean = np.median(a0_pred,axis=0)
idx1 = signal.find_peaks_cwt(a0_mean, np.arange(1,3))
idx2 = signal.find_peaks_cwt(-a0_mean, np.arange(1,3))

plt.figure(figsize=(10,4))
plt.plot(tnew[idx1],a0_mean[idx1],'b.',ms=1)
plt.plot(tnew[idx2],a0_mean[idx2],'b.',ms=1)

idx1 = signal.find_peaks_cwt(A_n_1h.values, np.arange(1,3))
idx2 = signal.find_peaks_cwt(-A_n_1h.values, np.arange(1,3))


plt.plot(A_n_1h.time[idx1],A_n_1h.values[idx1],'r.',ms=1)
plt.plot(A_n_1h.time[idx2],A_n_1h.values[idx2],'r.',ms=1)

plt.fill_between(tnew, np.percentile(a0_pred,2.5,axis=0),\
                 np.percentile(a0_pred,97.5,axis=0),color='0.2',alpha=0.5)

plt.ylim(-40,40)
plt.xlim(tnew[0],tnew[-1])
plt.grid(b=True)
plt.xlabel('Date [yyyy-mm]')
plt.ylabel('Amplitude [m]')
plt.tight_layout()

plt.savefig('../FIGURES/a0_harmonic_timeseries_{}.png'.format(sitename),dpi=150)
plt.savefig('../FIGURES/a0_harmonic_timeseries_{}.pdf'.format(sitename),dpi=150)

# Plot the power spectrum with the individual freqencies plus posterior distributions

In [ ]:
def power_spectra_w(tsec, u_r, window=np.hanning, K=3, power=2., axis=-1):
    """
    Calculates the power spectral density from a real valued quanity
    
    """
    
    M = tsec.shape[0]
    dt = tsec[1]-tsec[0]
    M_2 = int(np.floor(M/2))
      
    
    h_tk = window(M)
    # Weight the time-series and perform the fft
    u_r_t = u_r[...,:]*h_tk
    S_k = np.fft.fft(u_r_t, axis=axis)
    S = dt *np.abs(S_k)**power
    #S = np.mean(S_k,axis=-2)
        
    omega = np.fft.fftfreq(int(M),d=dt/(2*np.pi))
    
    #domega = 2*np.pi/(M*dt)
    domega = 1/(M*dt)
    
    # Extract the positive and negative frequencies
    omega_ccw = omega[0:M_2]
    #omega_cw = omega[M_2::] # negative frequencies
    S_ccw = S[...,0:M_2]
    #S_cw = S[...,M_2::]

    return omega_ccw,S_ccw,domega

In [ ]:
# Interpolate onto a regular time grid
#dt = 3600
dt = 60
timeinterp = pd.date_range(A_n_1h.time.values[0],A_n_1h.time.values[-1],freq='{}s'.format(dt))
A_n_1h_i = A_n.to_xray().interp({'time':timeinterp})
#A_n_1h_i = A_n_1h.interp({'time':timeinterp})


tsec = timeinterp.view(int)*1e-9

f, S, df = power_spectra(tsec, A_n_1h_i.values, K=1 )
fpred, Spred, dfpred = power_spectra(tsecnew, a0_pred[200,:], K=1 )

#f, S3, df = power_spectra(tsec, A_n_1h_i.values, K=3)

#dt=60
fcpd = f*86400/(2*np.pi)
fcpd2 = fpred*86400/(2*np.pi)

plt.figure()
plt.loglog(fcpd,S/(86400/(2*np.pi)), lw=0.5)
plt.loglog(fcpd2,Spred/(86400/(2*np.pi)), lw=0.5, alpha=0.5)

plt.xlim(1e-1,1e1)
# plt.xlim(1.8,2.2)

#plt.ylim(1e3,1e9)
plt.ylabel('Amplitude Spectrum [m$^2$ cpd$^{-1}$]')
plt.xlabel('Frequency [cpd]')
plt.grid(b=True,ls=':', which='both')
plt.tight_layout()

In [ ]:
# Calculate the power spectrum for the inferred harmonic parameters
psd = []
amp = []
for ii in range(len(omega)):
    A = trace['beta_{}_re'.format(ii)] + 1j*trace['beta_{}_im'.format(ii)]
    psd.append(np.abs( A*np.conj(A))*86400*dt)
    amp.append(np.abs(A))
    
psd = np.array(psd)
amp = np.array(amp)
psd_mu = np.median(psd,axis=1)
psd_low = np.percentile(psd,5,axis=1)
psd_high = np.percentile(psd,95,axis=1)

amp_mu = np.median(amp,axis=1)
amp_low = np.percentile(amp,5,axis=1)
amp_high = np.percentile(amp,95,axis=1)

omega_cpd = np.array([ff/(2*np.pi) for ff in omega])

yerr = np.stack([amp_mu - amp_low, amp_high-amp_mu])
#yerr = amp_high-amp_low
yerr.shape, yerr, amp_mu

In [ ]:
# Compute the fft amplitude directly
n = A_n_1h_i.values.shape[0]
#w = np.hamming(n)
#w = np.kaiser(n,8.6)
w= np.ones((n,))
wsum = 1/(np.sum(w)/n)
F = np.fft.fft(A_n_1h_i.values*w)
f = np.fft.fftfreq(n,dt)*2*np.pi # Hz
fcpd = f*86400/(2*np.pi)
#plt.figure()
#plt.plot(fcpd,2*np.abs(F)/n)

plt.figure()
ax = plt.subplot(111)
plt.plot(fcpd,2*np.abs(F)/n*wsum,'k',lw=1)
#plt.errorbar(omega_cpd, amp_mu, yerr=yerr, c='b',fmt='.')
plt.errorbar(omega_cpd, amp_mu, yerr=yerr, c='b', fmt='.', ms=3,ecolor='r')


plt.xlim(0.85,2.1)
#plt.xlim(1.85,2.05)
plt.ylim(1e-2,12.5)
plt.ylabel('Amplitude [m]')
plt.xlabel('Frequency [cpd]')

# inset axes....
axins = ax.inset_axes([0.05, 0.35, 0.57, 0.57])

axins.plot(fcpd,2*np.abs(F)/n*wsum,'k',lw=1)
#plt.plot(fcpd,2*np.sqrt(S3/dt)/(dt*2*np.pi),'k',lw=0.74)

axins.errorbar(omega_cpd, amp_mu, yerr=yerr, c='b', fmt='.', ms=3,ecolor='r')

# sub region of the original image
x1, x2, y1, y2 = 1.85,2.05, 0, 12
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
axins.text(1.89,3,'N2')
axins.text(1.935,11,'M2')
axins.text(2.0,5,'S2')
plt.tight_layout()
ax.indicate_inset_zoom(axins)

plt.savefig('../FIGURES/a0_spectra_amp_{}.png'.format(sitename),dpi=150)
plt.savefig('../FIGURES/a0_spectra_amp_{}.pdf'.format(sitename),dpi=150)